<a href="https://colab.research.google.com/github/supsi-dacd-isaac/TeachDecisionMakingUncertainty/blob/main/L03/Robust_Optimization_example_LP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rsome
!pip install mosek
!rm mosek.lic
!git clone https://github.com/roberock/mosek_lic
!cp ./mosek_lic/mosek.lic .
!rm -r ./mosek_lic
!mkdir -p /root/mosek
!cp ./mosek.lic /root/mosek
#!pip install -i https://pypi.gurobi.com gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.8/87.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 48.7 MB/s eta 0:00:00
rm: cannot remove 'mosek.lic': No such file or directory
Cloning into 'mosek_lic'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


In [ ]:
import rsome as rso
import numpy as np
from rsome import ro
from rsome import msk_solver as my_solver  #Import Mosek solver interface
#from rsome import grb_solver as my_solver  #Import Gurobi solver interface

In [ ]:
# Parameters setup
n  = 150
i = np.arange(1, n+1)
c  = 0.15+ i*0.05/150
a = np.zeros(n)
b = 0.02

# Describing the uncertainty
maxDev = 0.05/450 * (2*i*n*(n+1))**0.5
uBarPlus = maxDev
uBarMinus = -maxDev

## **Solving Deterministic Model**

Consider the following linear programming problem:
\begin{align}
\max\limits_{x}\;\;&c^\top x \\
\text{subject to}\;\;& a^\top x \leq b\\
& 0 \leq x \leq 1.
\end{align}

In [ ]:
#Create model
model = ro.Model('simpleExample_det')
x=model.dvar(n)
model.max(c@x)
model.st(a@x<=b)
model.st(x>=0)
model.st(x<=1)
model.solve(my_solver)
optobj_det = model.get()
xx_det   = x.get()
print('objective',optobj_det)
print('solution',xx_det)

Being solved by Mosek...
Solution status: Optimal
Running time: 0.0311s
objective 26.275
solution [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1.]


## **Solving Raw Robust Counterpart**

Consider the following robust linear optimization problem:
\begin{align}
\max\limits_{x}\;\;&c^\top x \\
\text{subject to}\\ \;\;& (a + u)^\top x \leq b && \forall u \in \mathcal{U}\\
& 0 \leq x \leq 1,
\end{align}
where $$\mathcal{U}:=\{u\;|\;\bar{u}^- \leq u \leq \bar{u}^+\}.$$

In [ ]:
#Create model
model = ro.Model('simpleExample_rawrobust')
x=model.dvar(n)

#Create uncertain vector
u = model.rvar(n)
#Create uncertainty set
USet = (u>=uBarMinus, u <= uBarPlus)
model.max(c@x)
#Robustify the constraint
model.st(((a+u)@x<=b).forall(USet))
model.st(x>=0)
model.st(x<=1)
model.solve(my_solver)

optobj_raw = model.get()
xx_raw   = x.get()

print('objective',optobj_raw)
print('solution',xx_raw)

Being solved by Mosek...
Solution status: Optimal
Running time: 0.0026s
objective 0.12713897073987465
solution [0.84571377 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0. 

## **Solving Reduced Robust Counterpart**

The robust counterpart can be reformulated as follows:

\begin{align}
\max\limits_{x,\lambda^+,\lambda^-}\;\;&c^\top x \\
\text{subject to}\\
\;\;& a ^\top x + (\bar{u}^+)^\top \lambda^+ - (\bar{u}^-)^\top \lambda^- \leq b, &&\\
& \lambda^+ -\lambda^- =x, \\
& \lambda^- \geq 0, \\
& \lambda^+ \geq 0,\\
& 0 \leq x \leq 1.
\end{align}

In [ ]:
#Create model
model = ro.Model('simpleExample_redrobust')
x=model.dvar(n)
#Create auxiliary variables
lambdaPlus=model.dvar(n)
lambdaMinus=model.dvar(n)

model.max(c@x)
#Modify the deterministic constraint
model.st(a@x + uBarPlus@lambdaPlus - uBarMinus@lambdaMinus <=b)
#Add constraints from dual representation of worst-case optimization
model.st(lambdaPlus-lambdaMinus == x)
model.st(lambdaPlus>=0)
model.st(lambdaMinus>=0)

model.st(x>= 0)
model.st(x<=1)

model.solve(my_solver)

optobj_red = model.get()
xx_red   = x.get()

print('objective',optobj_red)
print('solution',xx_red)

Being solved by Mosek...
Solution status: Optimal
Running time: 0.0017s
objective 0.12713897073987465
solution [0.84571377 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0. 